In [15]:
# Alex Cross and Kyle Kaminski

In [16]:
import random # Import random for the random seed generation
from PIL import Image # Import Image so we can use our image in this notebook
IMAGE_FILE = "lila.tiff"  # Name of our image (default picture from Dr. Baliga)

In [17]:
im = Image.open(IMAGE_FILE) # Store our image file into a variable

In [18]:
print(im.format, im.size, im.mode) # Print the file type, horizontal pixel count, vertical pixel count, and image type

TIFF (1000, 1333) RGBA


In [19]:
paraphrase = "Cryptography is awesome!"

In [20]:
secret = "This is our secret message. Kyle and I would very much like an A on this project!"

In [21]:
def flipbits(val,n):  # Flip n least significant bits in val
    return val^((1<<n)-1) # ^ is python's exclusive or operator

flipbits(181,1)  
# 17 = 10001 in binary, so if you flip 3 least significant bits, 
# you get 10110 which equals 22 in decimal

180

In [22]:
def findLSB(val): 
    return (val & 1) #bit AND with 1 to return the LSB value

findLSB(14)

0

In [23]:
def prepSecretMessage(secret): # Function for preparing the secret message
    byte_array = secret.encode() # Encode secret, store in a variable
    binary_int = int.from_bytes(byte_array, "big")
    binary_string = bin(binary_int).replace("0b","") #get rid of encoding header
    header = bin(len(binary_string)).replace("0b","").zfill(16) #create 16 bit header w/ 
                                                                #num of bits to find
    return header+binary_string
    

In [24]:
def hideMessage(im, paraphrase, secret): #takes an image file, the seed paraphrase, and the secret
    
    if len(''.join(format(ord(num), 'b') for num in secret)) <= 64000 : #check if message is appropriate size
        random.seed(paraphrase) #seed the generator
        px = im.load() #get pixels
        end_int = im.size[1]**2 # used as an end point for random int range
        preppedMess = prepSecretMessage(secret)
        pixel_set = {(0,0)} #keep track of (x,y) pairs
        i = 0 #iterator
        while i < len(preppedMess): #function should only run as long as there are bits to store
            rand = random.randrange(end_int)
            x = rand // im.size[0] #int division to find x
            y = rand % im.size[0] #mod to find y
            if (x,y) not in pixel_set and x <= im.size[0] and y <= im.size[1]: #safety checks
                pixel_set.add((x,y)) #add the usage to the set
                p = px[x,y]
                if findLSB(p[2]) != int(preppedMess[i]): #if the blue bit is not the same as the bit we need to input, change it.
                    b = flipbits(p[2],1)
                    px[x,y] = (p[0],p[1],b) #set the change 
                i = i+1
        im.save("newpic.tiff") #save the altered image, we will call this later on to prove this works
        im.close() # We don't need this anymore
    else:
        print ("Message size exceeds 8 KB") # We don't want a super long message

In [25]:
hideMessage(im, paraphrase, secret) # call the function

In [26]:
def convertMessage(messbin):
    binary_int = int(messbin, 2)
    byte_number = binary_int.bit_length() + 7 // 8
    binary_array = binary_int.to_bytes(byte_number, "big")
    ascii_text = binary_array.decode() # Decoding
    print(ascii_text)

In [27]:
def findMessage(im, paraphrase): #takes an image file and the paraphrase used to hide the message    
    random.seed(paraphrase) #seed the generator
    px = im.load() #get pixels
    end_int = im.size[1]**2 # used as an end point for random int range
    pixel_set = {(0,0)} #keep track of (x,y) pairs
    i = 0 #iterator
    header = '' #empty string
    while i < 16: #header is assumed to be zero-filled to 16 bits
        rand = random.randrange(end_int)
        x = rand // im.size[0] #int division to find x
        y = rand % im.size[0] #mod to find y
        if (x,y) not in pixel_set and x <= im.size[0] and y <= im.size[1]: #same safety checks to ensure same pixels get used
            pixel_set.add((x,y)) #add the usage to the set
            p = px[x,y]
            header = header+str(findLSB(p[2])) #rebuild the header
            i = i+1
    i = 0 #reset the iterator
    foundMessage = '' #empty string
    while i < int(header,2): #convert our binary to decimal and loop
        rand = random.randrange(end_int)
        x = rand // im.size[0] #int division to find x
        y = rand % im.size[0] #mod to find y
        if (x,y) not in pixel_set and x <= im.size[0] and y <= im.size[1]: #same safety checks to ensure same pixels get used
            pixel_set.add((x,y)) #add the usage to the set
            p = px[x,y]
            foundMessage = foundMessage+str(findLSB(p[2])) #rebuild the header
            i = i+1
    convertMessage(foundMessage)

In [28]:
findMessage(Image.open("newpic.tiff"), paraphrase)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      This is our secret message. Kyle and I would very much like an A on this project!
